Notebook dedicado a explorar los distintos usos que se le pueden dar a la función expr

In [2]:
val columns = Seq("name","age","weight", "height", "sex")
val data = Seq(
  ("Jose", 40, 74, 1.63, "M"), 
  ("María Isabel", 38, 58, 1.65, "F"), 
  ("Antonio", 27, 60, 1.68, "M"), 
  ("Norma", 63, 65, 1.54, "F")
)
val df = data.toDF(columns:_*)
df.show()

+------------+---+------+------+---+
|        name|age|weight|height|sex|
+------------+---+------+------+---+
|        Jose| 40|    74|  1.63|  M|
|María Isabel| 38|    58|  1.65|  F|
|     Antonio| 27|    60|  1.68|  M|
|       Norma| 63|    65|  1.54|  F|
+------------+---+------+------+---+



columns: Seq[String] = List(name, age, weight, height, sex)
data: Seq[(String, Int, Int, Double, String)] = List((Jose,40,74,1.63,M), (María Isabel,38,58,1.65,F), (Antonio,27,60,1.68,M), (Norma,63,65,1.54,F))
df: org.apache.spark.sql.DataFrame = [name: string, age: int ... 3 more fields]


In [3]:
import org.apache.spark.sql.functions._

// la función expr devuelve un elemento de tipo Columna, por lo cual puede utilizarse para mediante una expresión reemplazar 
// a una o un conjunto de funciones que devuelva una columna
df.select(expr("name"), col("age")).show()

+------------+---+
|        name|age|
+------------+---+
|        Jose| 40|
|María Isabel| 38|
|     Antonio| 27|
|       Norma| 63|
+------------+---+



import org.apache.spark.sql.functions._


In [4]:
df.select(expr("name"), expr("age")).show()

+------------+---+
|        name|age|
+------------+---+
|        Jose| 40|
|María Isabel| 38|
|     Antonio| 27|
|       Norma| 63|
+------------+---+



In [5]:
// Puede utilizarse para actualizar el valor de una columna, p.e. multiplicar el valor * 2
df.select(expr("name"), expr("age * 2").alias("age*2")).show()

+------------+-----+
|        name|age*2|
+------------+-----+
|        Jose|   80|
|María Isabel|   76|
|     Antonio|   54|
|       Norma|  126|
+------------+-----+



In [6]:
df.select(col("name"), isnan(col("age"))).show()

+------------+----------+
|        name|isnan(age)|
+------------+----------+
|        Jose|     false|
|María Isabel|     false|
|     Antonio|     false|
|       Norma|     false|
+------------+----------+



In [7]:
// Puede utilizarse para invocar funciones obteniendo el mismo resultado (comparar con la celda anterior)
df.select(col("name"), expr("isnan(age)")).show()

+------------+--------------------------+
|        name|isnan(CAST(age AS DOUBLE))|
+------------+--------------------------+
|        Jose|                     false|
|María Isabel|                     false|
|     Antonio|                     false|
|       Norma|                     false|
+------------+--------------------------+



In [8]:
// Puede utilizarse dentro de un groupby incluso en operaciones tan complejas
// como el ej. donde se agrupa por el resultado de la invocación de 2 funciones, obteniendo como resultado
// la agrapación por número de palabras que contengan un nombre
df.groupBy(expr("size(split(name, ' '))")).count().show()

+--------------------+-----+
|size(split(name,  ))|count|
+--------------------+-----+
|                   1|    3|
|                   2|    1|
+--------------------+-----+



In [9]:
// Se puede utilizar para indicar expresiones utilizadas en el filtrado de los datos
df.where(expr("sex = 'F'")).show()

+------------+---+------+------+---+
|        name|age|weight|height|sex|
+------------+---+------+------+---+
|María Isabel| 38|    58|  1.65|  F|
|       Norma| 63|    65|  1.54|  F|
+------------+---+------+------+---+



In [10]:
// Trabajar con selectExpr es similar a trabajar con la función expr
df.selectExpr("name", "age as edad", "abs(weight)").show()
df.select(expr("name"), expr("age as edad"), expr("abs(weight)")).show()

+------------+----+-----------+
|        name|edad|abs(weight)|
+------------+----+-----------+
|        Jose|  40|         74|
|María Isabel|  38|         58|
|     Antonio|  27|         60|
|       Norma|  63|         65|
+------------+----+-----------+

+------------+----+-----------+
|        name|edad|abs(weight)|
+------------+----+-----------+
|        Jose|  40|         74|
|María Isabel|  38|         58|
|     Antonio|  27|         60|
|       Norma|  63|         65|
+------------+----+-----------+



In [11]:
// En este caso sobrescribiendo una columna con un nuevo valor (la edad + 1)
df.withColumn("age", expr("age + 1")).show()

+------------+---+------+------+---+
|        name|age|weight|height|sex|
+------------+---+------+------+---+
|        Jose| 41|    74|  1.63|  M|
|María Isabel| 39|    58|  1.65|  F|
|     Antonio| 28|    60|  1.68|  M|
|       Norma| 64|    65|  1.54|  F|
+------------+---+------+------+---+



In [12]:
df.withColumn("age", col("age")+1).show()

+------------+---+------+------+---+
|        name|age|weight|height|sex|
+------------+---+------+------+---+
|        Jose| 41|    74|  1.63|  M|
|María Isabel| 39|    58|  1.65|  F|
|     Antonio| 28|    60|  1.68|  M|
|       Norma| 64|    65|  1.54|  F|
+------------+---+------+------+---+

